# Atomic Polar Tensor Explorer

**This script reads the EDIPOL file from the PLACZEK output and calculates all the atomic polar tensors. At the end, the derivatives are printed in an easy-to-read format.**

Remember, you can use this notebook as a tutorial to make your own program or you can just run it using your EDIPOL file.
Lets begin by impoting pandas to store our derivatives in DataFrames

In [15]:
import pandas as pd

Set the path to the EDIPOL file and open it using the *open* function. Remember to use *r* (read only) since you do not intend to modify the file. The *readlines()* method will transform your file in a list, where each entry is a line. 

In [4]:
file = 'MP2-EDIPOL'  # You can change the file path here
f=open(file, 'r').readlines()

"f" is a list containing all lines from EDIPOL. If you want to print the 19<sup>th</sup> for example: 

In [16]:
f[18] #remember: Python is a zero index array language

' ATOMIC DIPOLE MOMENT =        0.157673466010      -0.091028311095       0.000000429715\n'

We need to initialize some lists, which will be used to store temporary data.

In [5]:
atom = []
c = []
mx = []
my = []
mz = []
x = []
y = []
z = []

The first line of EDIPOL contains the displacement value. To calculate the atomic polar tensor, each atom is displaced in the positive and negative direction of each Cartesian axis.

In [6]:
delta_r = float(f[0].split()[2])

Next, read the atomic charges and dipoles from the equilibrium position. 

In [6]:
for line in f:
    if 'EDIPOL-EQ-GEOM:' in line: # search for this keyword - start here
        start = f.index(line)
    if 'EDIPOL+' in line:   # next EDIPOL keyword  - end here
        end = f.index(line)
        break
for i in range (start, end, 1):
    if 'ATOM ' in f[i]:
        atom.append(f[i].split()[1] + '_' + f[i].split()[2]) # read atoms' names and labels
    if 'ATOMIC CHARGE =' in f[i]:
        c.append(float(f[i].split()[3]))  # read atomic chages
    if 'ATOMIC DIPOLE MOMENT =' in f[i]:
        mx.append(float(f[i].split()[4])) # read atomic dipole in x
        my.append(float(f[i].split()[5])) # read atomic dipole in y
        mz.append(float(f[i].split()[6])) # read atomic dipole in z
 

Now, we move the properties to a pandas DataFrame.

In [7]:
eq_prop = pd.DataFrame(columns=['charge', 'mx', 'my', 'mz'], index=atom)
eq_prop['charge'] = c
eq_prop['mx'] = mx #mx = atomic dipole in x direction
eq_prop['my'] = my #my = atomic dipole in x direction
eq_prop['mz'] = mz #mz = atomic dipole in x direction

Lets read the atomic geometry. we need it to calculate the dipole from charges. 
Search for the "MOLECULAR GEOMETRY" keyword in the EDIPOL file. 

In [8]:
for line in f:
    if 'MOLECULAR GEOMETRY' in line: #Keyword -- start here
        start = f.index(line)

for i in range(start+2, len(f), 1):
    if len(f[i].split()) >= 3:
        x.append(float(f[i].split()[1]))
        y.append(float(f[i].split()[2]))
        z.append(float(f[i].split()[3]))

eq_geo = pd.DataFrame(columns=['x', 'y', 'z'], index=atom) #remember to create a dataframe
eq_geo['x'] = x
eq_geo['y'] = y
eq_geo['z'] = z

It's time to read the charges and dipole for the non-equilibrium geometries. for each of them, create a DataFrame and append it to a list, in which the elements are organized following:
<ol start="0">
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **+X** direction</li>
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **-X** direction</li>
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **+Y** direction</li>
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **-Y** direction</li>
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **+Z** direction</li>
 <li> Electronics properties from the geometry where **ATOM_1** is displaced in **-Z** direction</li>
 <li>. Electronics properties from the geometry where **ATOM_2** is displaced in **+X** direction</li>
 <li> ...</li>
</ol>

In [9]:
displaced_geo = []
displacements = ['+X', '-X', '+Y', '-Y', '+Z', '-Z'] #List of displacements

Read the properties the same way we did for the equilibrium geometry, just changing the keywords.  

In [10]:
for a in atom:
    for coord in displacements:
        c = [] #reset the list at the beggining of each iteration
        mx = []
        my = []
        mz = []       
        for line in f:
            if 'EDIPOL'+coord+a in line: # EDIPOL +/- X/Y/Z - Start here
                start = f.index(line)
        for i in range(start+2, start+2 + len(atom)*4):
            if 'MONOPOLE' in f[i]: #Read the atomic MONOPOLE MOMENT (A.K.A. atomic CHARGE)
                c.append(float(f[i].split()[3]))
            if 'DIPOLE' in f[i]:
                mx.append(float(f[i].split()[3]))
                my.append(float(f[i].split()[4]))
                mz.append(float(f[i].split()[5]))
        
        data = pd.DataFrame(columns=['charge', 'mx', 'my', 'mz'], index=atom) # Save to DataFrame 
        data['charge'] = c
        data['mx'] = mx
        data['my'] = my
        data['mz'] = mz
        displaced_geo.append(data) # Append DataFrame to lsit. 

We have collected all data we need to calculate the derivatives.
### CALCULATING THE NUMERIAL DERIVATIVE ###
for a property $n$, the numerical derivative is given by: $$\frac{dn}{dr} = \frac{n^+-n^-}{2*delta\_r}$$ 
where $delta\_r$ is the displacement value, $n^+$ and $n^-$ are the value of the property when the atom are displaced in the postive and negative direction of coordinate $r$

### The dipole moment derivative ###
Consider the dipolo moment of a molecule as:
$$\vec{\rho} = \rho_x\hat{i} + \rho_y\hat{j} + \rho_z\hat{k}$$




In [11]:
#calcular as derivadas, centralizando no carbono
## centralizar no carbono
geo_center = eq_geo-eq_geo.loc['C_1']
d_temp= []
for i in range(0, len(displaced_geo), 2):
    temp = 0.529177*(displaced_geo[i] - displaced_geo[i+1])/(2*delta_r)
    temp['charge_x'] = geo_center['x']*temp['charge']/0.529177
    temp['charge_y'] = geo_center['y']*temp['charge']/0.529177
    temp['charge_z'] = geo_center['z']*temp['charge']/0.529177
    del temp['charge']
    d_temp.append(temp)

In [12]:
##Criar dicionario
cart = ['X_', 'Y_', 'Z_']
terms = [i+j for j in atom for i in cart]

derivatives = {terms[i]:d_temp[i] for i in range(len(terms))}

In [14]:
derivatives['X_C_1']

,mx,my,mz,charge_x,charge_y,charge_z
C_1,0.731196,0.000256,-0.033857,0.000000,0.000000,0.000000
H_2,-0.194712,-0.004699,-0.000450,0.000000,0.000000,0.001828
H_3,-0.051621,0.000005,0.000003,0.000000,-0.000037,0.000000
H_4,-0.033504,-0.010553,-0.000007,-0.303564,0.175263,0.000000
H_5,-0.033482,0.010531,0.000031,-0.303524,-0.175240,-0.000000
H_6,-0.199370,0.002112,0.000464,0.000000,0.000000,-0.002132
